In [1]:
import numpy as np
from bokeh.plotting import figure, show
from bokeh.layouts import column
from bokeh.models import DataTable, TableColumn, PointDrawTool, ColumnDataSource, TextAreaInput
from bokeh.io import output_notebook
output_notebook()

Loading BokehJS ...

In [2]:
N = 50
M = 40
L = 30
xx = np.arange(N, dtype=np.float64)
yy = np.arange(M, dtype=np.float64)
zz = np.arange(L, dtype=np.float64)
x, y, z = np.meshgrid(xx, yy, zz, indexing='ij')
b = N/20.0
c = M/2.0
d = L/2.0
r = np.sqrt(((x-c)/b)**2 + ((y-c)/b)**2 + ((z-d)/b)**2)
a = np.sin(r)
a.shape

(50, 40, 30)

In [3]:
class Plotter:
    def __init__(self, data=None):
        self.doc = None
        self.renderer = None
        self.data = data

    def profile_handler(self, attr, old, new):
        if len(new["x"]) > len(old["x"]):
            ind = -1
            new_line = True
        else:
            ind = np.argmax(np.array(old['x']) != np.array(new['x']))
            new_line = False
        dx = 40.0 / 20.0
        dy = 50.0 / 15.0
        i = int(new['x'][ind] / dx)
        j = int(new['y'][ind] / dy)
        s1 = "{}, {}, {}, {}, {}".format(ind, i, j, new['x'][ind], dx)
        s2 = str(old)
        s3 = str(new)
        s4 = str(old['x'] != new['x'])
        self.text_area_input.value = "{}\n{}\n{}\n{}".format(s1, s2, s3, s4)
        if new_line:
            self.lines.append(self.profile.line(np.arange(self.data.shape[-1]), self.data[j, i, :],
                                   color="#8888cc", line_width=1.5, alpha=0.8))
        else:
            self.lines[ind].data_source.data['y'] = self.data[j, i, :]

        
    def modify_doc(self, doc):
        self.doc=doc
#         p = figure(title='scope')
#         p.background_fill_color = 'lightgrey'

        p = figure(
           plot_width=500, plot_height=400)
        p.x_range.range_padding = p.y_range.range_padding = 0

#         # must give a vector of image data for image parameter
        self.image_renderer = p.image(image=[self.data[..., 0]], x=0, y=0, dw=20, dh=15,
                                      palette="Viridis256", level="image")

#         source = ColumnDataSource({
#             'x': self.scatter[:, 0], 'y': self.scatter[:, 1],
# #             'color': ['red', 'green', 'yellow']
#         })
    
        self.source = ColumnDataSource({
            'x': [], 'y': [],
#             'color': ['red', 'green', 'yellow']
        })

#         self.renderer = p.scatter(x='x', y='y', source=source, color='color', size=10)
        self.renderer = p.scatter(x='x', y='y', source=self.source, size=10)
#         columns = [TableColumn(field="x", title="x"),
#                    TableColumn(field="y", title="y"),
#                    TableColumn(field='color', title='color')]
#         self.table = DataTable(source=self.source, columns=columns, editable=True, height=200)


#         text_input = TextInput(value="default", title="Label:")
        self.source.on_change("data", self.profile_handler)
        
        draw_tool = PointDrawTool(renderers=[self.renderer], empty_value='black')
        p.add_tools(draw_tool)
#         p.toolbar.active_tap = draw_tool

        
        self.profile = figure(plot_width=500, plot_height=250)
        self.lines = []
        
        self.text_area_input = TextAreaInput(value="default", rows=6, title="Label:")
        
        self.doc.add_root(column(p, self.profile, self.text_area_input))
#         self.doc.add_root(column(p, self.text_area_input))
#         self.doc.add_root(column(p))

In [4]:
p = Plotter(data=a)

In [6]:
show(p.modify_doc, notebook_url="localhost:8891")

In [ ]:
p.renderer.data_source.data

In [ ]:
p.table.source.data

In [ ]:
x = np.array([1, 2, 3, 4])
y = np.array([1, 2, 6, 4])
np.argmax(x != y)

In [ ]:
x != y